In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import joblib

BASE_DIR      = Path(r"C:\Users\islem\Desktop\R37-LightGBM")
RAW_DATA_DIR  = BASE_DIR / "data" / "raw"
PROC_DATA_DIR = BASE_DIR / "data" / "processed"
MODELS_DIR    = BASE_DIR / "models" / "lgbm_baseline"
GEOM_OUT_DIR  = BASE_DIR / "geometry_auto"
GEOM_OUT_DIR.mkdir(exist_ok=True)

EDP_PATH   = RAW_DATA_DIR / "edp.csv"
DATA_PATH  = PROC_DATA_DIR / "processed_data_multi.npz"
MODEL_PATH = MODELS_DIR / "model.pkl"

edp = pd.read_csv(EDP_PATH)
data = np.load(DATA_PATH, allow_pickle=True)
X = data["X"]; feature_cols = data["feature_cols"]; target_cols = data["target_cols"]
model = joblib.load(MODEL_PATH)

g1 = np.load(RAW_DATA_DIR / "geometries_1.npy")
g2 = np.load(RAW_DATA_DIR / "geometries_2.npy")
geometries = np.concatenate([g1, g2], axis=0)

print("EDP:", edp.shape, "X:", X.shape, "geometries:", geometries.shape)


EDP: (2000, 53) X: (2000, 36) geometries: (2000, 2, 9, 112, 4)


In [2]:
y_pred_all = model.predict(X)  # (2000, 6)

df_pred = pd.DataFrame(y_pred_all, columns=target_cols)
df_all  = pd.concat([edp, df_pred.add_prefix("pred_")], axis=1)

# مثال score: كفاءة isen - 0.1 * |loss|
score = df_all["pred_computation_pout_C_ISEN_EFF_TOT2TOT"] \
        - 0.1 * df_all["pred_computation_pout_C_PRES_LOSS"].abs()

df_all["score"] = score
df_all_sorted = df_all.sort_values("score", ascending=False)


In [3]:
N_TOP = 10
top_designs = df_all_sorted.head(N_TOP).copy()
top_indices = top_designs.index.values  # هذه هي مفاتيح geometries

top_designs[["score"] + list(df_pred.columns)].head()


,score,computation_pout_C_ISEN_EFF_TOT2TOT,computation_pout_C_POLY_EFF_TOT2TOT,computation_pout_TOT_PRES_RATIO,computation_pout_MASS_FLOW,computation_pout_TORQUE,computation_pout_C_PRES_LOSS
277,0.358382,0.866206,0.877351,1.8608,20.746,749.24,-5.074349
20,0.353973,0.859564,0.871280,1.8629,20.807,758.29,-5.048774
1187,0.353208,0.851334,0.863061,1.7974,20.291,700.64,-4.973868
1372,0.352865,0.862582,0.874229,1.8826,20.922,774.27,-5.090755
1660,0.352554,0.857641,0.869477,1.8588,20.763,755.56,-5.051596


In [4]:
from pathlib import Path

def export_geometry_for_design(idx, base_out_dir):
    geom = geometries[idx]          # (2, 9, 112, 4)
    suct = geom[0, :, :, :3]
    press = geom[1, :, :, :3]

    design_dir = base_out_dir / f"design_{idx:04d}"
    design_dir.mkdir(parents=True, exist_ok=True)

    # مثال مبسط: حفظ المصفوفات كنقاط numpy لتستخدمها سكربت خارجي
    np.save(design_dir / "suction.npy", suct)
    np.save(design_dir / "pressure.npy", press)

    # يمكن لاحقًا إضافة كتابة profile.curve / hub.curve / shroud.curve هنا
    return design_dir

for idx in top_indices:
    ddir = export_geometry_for_design(idx, GEOM_OUT_DIR)
    print("Exported geometry for design", idx, "->", ddir)


Exported geometry for design 277 -> C:\Users\islem\Desktop\R37-LightGBM\geometry_auto\design_0277
Exported geometry for design 20 -> C:\Users\islem\Desktop\R37-LightGBM\geometry_auto\design_0020
Exported geometry for design 1187 -> C:\Users\islem\Desktop\R37-LightGBM\geometry_auto\design_1187
Exported geometry for design 1372 -> C:\Users\islem\Desktop\R37-LightGBM\geometry_auto\design_1372
Exported geometry for design 1660 -> C:\Users\islem\Desktop\R37-LightGBM\geometry_auto\design_1660
Exported geometry for design 658 -> C:\Users\islem\Desktop\R37-LightGBM\geometry_auto\design_0658
Exported geometry for design 1136 -> C:\Users\islem\Desktop\R37-LightGBM\geometry_auto\design_1136
Exported geometry for design 1991 -> C:\Users\islem\Desktop\R37-LightGBM\geometry_auto\design_1991
Exported geometry for design 1228 -> C:\Users\islem\Desktop\R37-LightGBM\geometry_auto\design_1228
Exported geometry for design 1835 -> C:\Users\islem\Desktop\R37-LightGBM\geometry_auto\design_1835
